In [ ]:
import boto3
from owlna import Athena
athena = Athena(session=boto3.Session())
athena

In [ ]:
with athena.connect() as connection:
    # persists boto3 client
    boto3_client = connection.client
    
    # get table
    table = connection.table(catalog='AwsDataCatalog', database='unittest', name='pyathena_unittest')
    
    # insert arrow    
    import pyarrow

    data = pyarrow.RecordBatch.from_arrays(
        [
            pyarrow.array([None, "pstring"]),
            pyarrow.array([None, 1]),
            pyarrow.array([None, "test"]),
            pyarrow.array([None, 10]),
            pyarrow.array(["2022-10-10 12:12:00.123Z", "2022-10-10 12:12:00.123456789Z"]),
            pyarrow.array(["2022-10-10", "2022-10-12"])
        ],
        schema=pyarrow.schema(
            [
                pyarrow.field("pstring", pyarrow.string()),
                pyarrow.field("pint", pyarrow.int64()),
                pyarrow.field("string", pyarrow.string()),
                pyarrow.field("int", pyarrow.int32()),
                pyarrow.field("timestamp", pyarrow.string()),
                pyarrow.field("date", pyarrow.string())
            ]
        )
    )
    
    # cf https://arrow.apache.org/docs/python/generated/pyarrow.dataset.write_dataset.html#pyarrow.dataset.write_dataset
    
    # existing_data_behavior = "overwrite_or_ignore" default will append
    # existing_data_behavior = "delete_matching" to overwrite dir / partition dir
    
    table.insert_arrow_batch(
        data,
        base_dir=None, # put {"key": "value"} to append to table.location path
        existing_data_behavior="overwrite_or_ignore",
        safe=False # safe casting to table.schema_arrow, default owlna.config.DEFAULT_SAFE_MODE
    )

In [ ]:
with athena.connect() as connection:
    # list tables
    for table in connection.tables(
        catalog='AwsDataCatalog', database='mad',
        expression='*' # regex expression to match with name
    ):
        print(table.name)
        print(table.schema_arrow)